*Step.1 導入相關套件*

In [ ]:
!pip install datasets

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

*Step.2 載入數據*

In [ ]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

In [ ]:
ds[:3]

*Step.3 數據前處理*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

In [9]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

*Step.4 模型建立*

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")

In [12]:
sum(param.numel() for param in model.parameters())

1303111680

model size: 1.3B

model: 1.3G * 4 ~= 5.2G

gradient: 1.3G * 4 ~= 5.2G

optimizer: 1.3G * 4 * 2 ~= 10.4G

sum: 20.8G

# Prompt tuning
# PEFT Step.1 配置文件

In [ ]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

# Soft Prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
#                num_virtual_tokens=10,
#                encoder_reparameterization_type = PromptEncoderReparameterizationType.LSTM,
#                encoder_dropout = 0.1,
#                encoder_num_layers = 5,
#                encoder_hidden_size = 1024))
# config
# Hard Prompt
config = PromptTuningConfig(task_type = TaskType.CAUSAL_LM,
               prompt_tuning_init = PromptTuningInit.TEXT,
               prompt_tuning_init_text = "下面是一段人與機器人的對話。",
               num_virtual_tokens = len(tokenizer("下面是一段人與機器人的對話。")["input_ids"]),
               tokenizer_name_or_path = "Langboat/bloom-1b4-zh")
config

# PEFT Step.2 建立模型

In [14]:
model = get_peft_model(model, config)

In [ ]:
model

In [16]:
model.print_trainable_parameters()

trainable params: 22,528 || all params: 1,303,134,208 || trainable%: 0.0017


*Step.5 訓練參數*

In [ ]:
args = TrainingArguments(
    output_dir = "./chatbot",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    logging_steps = 10,
    num_train_epochs = 1,
    save_steps = 20,
)

*Step.6 訓練器*

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset = tokenized_ds,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

*Step.7 模型訓練*

In [ ]:
trainer.train()

# 載入訓練好的PEFT模型

In [ ]:
from peft import PeftModel

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
peft_model = PeftModel.from_pretrained(model=model, model_id="./chatbot/checkpoint-20/")

*Step.8 模型推理*

In [ ]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考試有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))